In [1]:
import geopandas as gpd
import shapely
from shapely.geometry import box, LineString, Point,MultiPoint
from shapely.ops import nearest_points
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import momepy
import os
import networkx as nx
import re
from IPython.core.interactiveshell import InteractiveShell
pd.set_option('display.max_rows', None)
InteractiveShell.ast_node_interactivity = "all"

/Applications/anaconda3/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
cwd = os.getcwd()
cwd

'/Users/lindsaygraff/Documents/Multimodal Transit Research/Code'

### GTFS Data

In [3]:
# https://transitfeeds.com/p/port-authority-of-allegheny-county/274
# or, https://www.portauthority.org/business-center/developer-resources/
# Change directory to location of data
new_dir = cwd.replace('Code','Data/PortAuthority/GTFS/')
new_dir
os.chdir(new_dir)

'/Users/lindsaygraff/Documents/Multimodal Transit Research/Data/PortAuthority/GTFS/'

**Read Data**

In [4]:
stops_df = pd.read_csv('stops.txt')
stops_df.head()
stop_times_df = pd.read_csv('stop_times.txt')
stop_times_df.head()
trips_df = pd.read_csv('trips.txt')
trips_df.head()
cal_dates_df = pd.read_csv('calendar_dates.txt')
cal_dates_df.head()
cal_df = pd.read_csv('calendar.txt')

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone,wheelchair_boarding
0,4875,4875,ARLINGTON AVE + FREDERICK,NaN,40.417694,-79.982614,01,NaN,NaN,NaN,NaN,0
1,4876,4876,ARLINGTON AVE + KOEHLER,NaN,40.417637,-79.984507,01,NaN,NaN,NaN,NaN,0
2,4877,4877,ARLINGTON AVE + S 18TH,NaN,40.417582,-79.986232,01,NaN,NaN,NaN,NaN,0
3,4884,4884,WARRINGTON AVE + ALLEN,NaN,40.421845,-79.993579,01,NaN,NaN,NaN,NaN,0
4,4886,4886,WARRINGTON AVE + BELTZHOOVER,NaN,40.421712,-79.997246,01,NaN,NaN,NaN,NaN,0


<ipython-input-4-10ca047c2a1f>:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  stop_times_df = pd.read_csv('stop_times.txt')


,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint
0,6514020,15:49:04,15:49:04,1188,48,64 I LAWRENCEVILLE VIA SQ HILL-SHADYSIDE-BLOOM...,0,0,13131.21,0
1,6514020,15:49:41,15:49:41,1189,49,64 I LAWRENCEVILLE VIA SQ HILL-SHADYSIDE-BLOOM...,0,0,13247.89,0
2,6514020,15:50:59,15:50:59,18148,50,64 I LAWRENCEVILLE VIA SQ HILL-SHADYSIDE-BLOOM...,0,0,13584.90,0
3,6514020,15:52:00,15:52:00,14852,51,64 I LAWRENCEVILLE VIA SQ HILL-SHADYSIDE-BLOOM...,0,0,13700.54,1
4,6514020,15:52:50,15:52:50,14853,52,64 I LAWRENCEVILLE VIA SQ HILL-SHADYSIDE-BLOOM...,0,0,13885.95,0


,trip_id,route_id,service_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed
0,10517020,61C,3,Downtown,NaN,1,176586102.0,shp-61C-06,1,1
1,8275010,71D,4,Hamilton,NaN,0,176856101.0,shp-71D-52,1,0
2,4380020,P1,3,East Busway,NaN,0,176377502.0,shp-P1-53,1,1
3,6302020,51,3,Downtown,NaN,1,176573502.0,shp-51-18,1,0
4,13430020,14,3,Ambridge,NaN,0,176726002.0,shp-14-54,1,1


,service_id,date,exception_type
0,2,20220530,1
1,3,20220530,2
2,2288,20220530,1
3,2289,20220530,2
4,W,20220530,2


## Data Exploration

In [5]:
cal_df.service_id.unique()
cal_df
cal_df.service_id.dtypes
# we will use service_id in ['W', '3'] when joining with trips_df because this id defines weekday service

array(['1', '2', '3', '4', '2287', '2288', '2289', '2290', 'S', 'W', 'U'],
      dtype=object)

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,1,0,0,0,0,0,1,0,20220424,20220625
1,2,0,0,0,0,0,0,0,20220424,20220625
2,3,1,1,1,1,1,0,0,20220424,20220625
3,4,0,0,0,0,0,0,1,20220424,20220625
4,2287,0,0,0,0,0,1,0,20220424,20220625
5,2288,0,0,0,0,0,0,0,20220424,20220625
6,2289,1,1,1,1,1,0,0,20220424,20220625
7,2290,0,0,0,0,0,0,1,20220424,20220625
8,S,0,0,0,0,0,1,0,20220424,20220625
9,W,1,1,1,1,1,0,0,20220424,20220625


dtype('O')

In [6]:
# Explore data for better understand
trips_subset = trips_df.loc[trips_df['route_id'] == '61C'][['trip_id', 'route_id', 'service_id', 'direction_id']]
trips_subset_in = trips_subset.loc[trips_subset['direction_id'] == 1]
trips_subset_in.head(3)
trips_subset_in.shape
trips_subset_in['trip_id'].unique().shape
trips_df.shape
trips_df['trip_id'].unique().shape
trips_df.service_id.unique()

# we can see that the trips df is uniquely defined by a trip ID. one route is associated with many trips
# think: 61C is a route. it passes through Forbes/Morewood many times. each cycle is a trip

,trip_id,route_id,service_id,direction_id
0,10517020,61C,3,1
165,13210080,61C,2,1
393,6709080,61C,2,1


(170, 4)

(170,)

(16567, 10)

(16567,)

array(['3', '4', '1', '2', '2287', '2288', '2289', '2290', 'S', 'U', 'W'],
      dtype=object)

In [7]:
a = stop_times_df.loc[stop_times_df['trip_id'] == 6514020]  # this trip is associated with 58 stops (64 route)
a.head()
a.shape 
a.stop_id.unique().shape
e = stop_times_df.loc[stop_times_df['trip_id'] == 10517020] # this trip is associated with 87 stops (61C route)
e.head()
e.shape
# as expected, a single trip of the same route passes through the same number of stops

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint
0,6514020,15:49:04,15:49:04,1188,48,64 I LAWRENCEVILLE VIA SQ HILL-SHADYSIDE-BLOOM...,0,0,13131.21,0
1,6514020,15:49:41,15:49:41,1189,49,64 I LAWRENCEVILLE VIA SQ HILL-SHADYSIDE-BLOOM...,0,0,13247.89,0
2,6514020,15:50:59,15:50:59,18148,50,64 I LAWRENCEVILLE VIA SQ HILL-SHADYSIDE-BLOOM...,0,0,13584.90,0
3,6514020,15:52:00,15:52:00,14852,51,64 I LAWRENCEVILLE VIA SQ HILL-SHADYSIDE-BLOOM...,0,0,13700.54,1
4,6514020,15:52:50,15:52:50,14853,52,64 I LAWRENCEVILLE VIA SQ HILL-SHADYSIDE-BLOOM...,0,0,13885.95,0


(58, 10)

(58,)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint
248510,10517020,12:32:00,12:32:00,22902,1,61C I MCKEESPORT HOMESTEAD PITTSBURGH VIA OAKLAND,1,1,0.00,1
248511,10517020,12:33:00,12:33:00,22899,2,61C I MCKEESPORT HOMESTEAD PITTSBURGH VIA OAKLAND,0,1,198.52,1
248512,10517020,12:33:52,12:33:52,9892,3,61C I MCKEESPORT HOMESTEAD PITTSBURGH VIA OAKLAND,0,0,576.24,0
248513,10517020,12:34:15,12:34:15,9894,4,61C I MCKEESPORT HOMESTEAD PITTSBURGH VIA OAKLAND,0,0,767.76,0
248514,10517020,12:34:37,12:34:37,18174,5,61C I MCKEESPORT HOMESTEAD PITTSBURGH VIA OAKLAND,0,0,988.03,0


(87, 10)

In [8]:
b = trips_df.loc[trips_df['trip_id'] == 6514020 ]  # only one row bc trips_df is uniquely defined by trip_id
b.head()
d = trips_df.loc[trips_df['trip_id'] == 1312020 ]  # see above
d.head()
 # number of unique trip ids associated with route_id == 64 and weekday service
c = trips_df.loc[((trips_df['route_id'] == '64') & (trips_df['service_id'].isin(['W','3'])))] 
c.head()
c.shape
c.trip_id.unique().shape
c.service_id.unique()

,trip_id,route_id,service_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed
3659,6514020,64,3,Lawrenceville,NaN,1,176551702.0,shp-64-03,1,1


,trip_id,route_id,service_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed
13872,1312020,64,3,Lawrenceville,NaN,1,176555002.0,shp-64-03,1,1


,trip_id,route_id,service_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed
22,936020,64,3,Lawrenceville,NaN,1,176554902.0,shp-64-03,1,1
764,12510020,64,3,Lawrenceville,NaN,1,176588502.0,shp-64-03,1,1
798,7640020,64,3,Waterfront,NaN,0,176555202.0,shp-64-51,1,0
866,1766020,64,3,Waterfront,NaN,0,176555202.0,shp-64-51,1,0
1018,6492020,64,3,Lawrenceville,NaN,1,176555102.0,shp-64-03,1,1


(80, 10)

(80,)

array(['3'], dtype=object)

Difference between trip ID and route ID
https://stackoverflow.com/questions/31659286/why-does-a-route-have-different-trip-ids-gtfs

In [9]:
trips_df.dtypes

trip_id                    int64
route_id                  object
service_id                object
trip_headsign             object
trip_short_name          float64
direction_id               int64
block_id                 float64
shape_id                  object
wheelchair_accessible      int64
bikes_allowed              int64
dtype: object

## Testing with a single (route, direction) pair
**Goal: for the given pair, find the sequence of points that defines the pair**

Let's consider the 64 bus i.e. route_id == '64'
There are two directions: direction_id == 1 and direction_id == 0
For each (route_id, direction_id) pair, there are many service_ids (because different schedule depending on DoW) and different trip_ids (because one single day of week has many frequencies)

In [10]:
trips_filtered = trips_df.loc[(trips_df['route_id'] == '64') & (trips_df['direction_id'] == 1) & (trips_df['service_id'] == '3')].reset_index()
len(trips_filtered)  # there are 40 trips meetin this criteria
trips_filtered.head()

40

,index,trip_id,route_id,service_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed
0,22,936020,64,3,Lawrenceville,NaN,1,176554902.0,shp-64-03,1,1
1,764,12510020,64,3,Lawrenceville,NaN,1,176588502.0,shp-64-03,1,1
2,1018,6492020,64,3,Lawrenceville,NaN,1,176555102.0,shp-64-03,1,1
3,1738,7733020,64,3,Lawrenceville,NaN,1,176591002.0,shp-64-03,1,1
4,1795,3474020,64,3,Lawrenceville,NaN,1,176396302.0,shp-64-03,1,1


In [11]:
# arbitrarily choose the first entry
single_trip = trips_filtered.loc[0].to_frame().transpose()
single_trip['trip_id'] = single_trip['trip_id'].astype('int64')
single_trip
#len(stop_times_df.loc[stop_times_df['trip_id'].isin([936020])])
# join trips_filtered with stop_times_df on trip_id
temp = single_trip.merge(stop_times_df, on='trip_id', how='left').sort_values(by = ['stop_sequence'], ascending=True)
#temp.head(2)
temp.shape  # 58 stops, as expected
temp['stop_id'] = temp['stop_id'].astype('str')
stops_df['stop_id'] = stops_df['stop_id'].astype('str')
stops_trips_df = temp.merge(stops_df, on='stop_id', how='inner')  # 58 stops, as expected
stops_trips_df.shape
cols_keep = ['trip_id', 'route_id', 'service_id', 'direction_id', 'stop_sequence', 'stop_lat', 'stop_lon']
stops_trip_df = stops_trips_df[cols_keep]
# convert to gdf and create Point column
stops_trip_gdf = gpd.GeoDataFrame(stops_trip_df, geometry=gpd.points_from_xy(stops_trip_df.stop_lon, stops_trip_df.stop_lat)).drop(columns=['stop_lat','stop_lon'])

,index,trip_id,route_id,service_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed
0,22,936020,64,3,Lawrenceville,NaN,1,176554902.0,shp-64-03,1,1


(58, 20)

(58, 31)

In [12]:
# project to a single columns
route_direction_points = [{'route_id': stops_trip_gdf['route_id'].unique()[0], 
                          'direction_id': stops_trip_gdf['direction_id'].unique()[0],
                          'point_sequence': stops_trip_gdf['geometry'].tolist()}]
route_direction_points_df = pd.DataFrame.from_records(data=route_direction_points) #, columns=['route_id','direction_id','point_sequence'])

### Now generalize to the full dataset
**Goal is to build df with the following columns: route_id, direction_id, point_sequence**

In [13]:
# join trips with stop times
temp = trips_df.merge(stop_times_df, on='trip_id', how='left').sort_values(by = ['stop_sequence'], ascending=True)

In [14]:
temp.head(2)
temp.shape  
temp['stop_id'] = temp['stop_id'].astype('str')

,trip_id,route_id,service_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint
0,10517020,61C,3,Downtown,NaN,1,176586102.0,shp-61C-06,1,1,12:32:00,12:32:00,22902,1,61C I MCKEESPORT HOMESTEAD PITTSBURGH VIA OAKLAND,1,1,0.0,1
492780,13721070,51,1,Carrick - West Mifflin,NaN,0,176466807.0,shp-51-64,1,0,14:25:00,14:25:00,4833,1,51 OM CARRICK-SOUTH SIDE TO WEST MIFFLIN,0,0,0.0,1


(927771, 19)

In [15]:
cols_keep = ['trip_id', 'route_id', 'service_id', 'direction_id', 'stop_id', 'stop_sequence', 'stop_lat', 'stop_lon']
# join to stops_df, which contains physical location of stops
stops_trips_df = temp.merge(stops_df, on='stop_id', how='inner')[cols_keep] 
stops_trips_df.shape  # should have same number of rows as before
stops_trips_df.head()

(927771, 8)

,trip_id,route_id,service_id,direction_id,stop_id,stop_sequence,stop_lat,stop_lon
0,10517020,61C,3,1,22902,1,40.352322,-79.860598
1,1959020,61C,3,1,22902,1,40.352322,-79.860598
2,7848070,61C,1,1,22902,1,40.352322,-79.860598
3,1116010,61C,4,1,22902,1,40.352322,-79.860598
4,11433020,P7,3,1,22902,1,40.352322,-79.860598


In [16]:
# filter the df by service_id as we are only considering weekday service
stops_trips_df = stops_trips_df.loc[stops_trips_df['service_id'].isin(['3','W'])]
# get a list of trip_ids to include: arbitrarily the first trip for a given (route_id, direction_id) pair
trip_id_keep = stops_trips_df.groupby(['route_id', 'direction_id']).first()['trip_id'].tolist()
# now filter by this list
stops_trip_df = stops_trips_df.loc[stops_trips_df['trip_id'].isin(trip_id_keep)].sort_values(by=['route_id', 'direction_id','service_id', 'stop_sequence'])
stops_trip_df.shape

(10436, 8)

In [17]:
# let's check this against our 64, dir=1 results from above
test64 = stops_trip_df.loc[(stops_trip_df['route_id'] == '64') & (stops_trip_df['direction_id'] == 1)]
test64.shape  # 58 stops, as above

(58, 8)

In [52]:
# convert to gdf and create Point column
stops_trip_gdf = gpd.GeoDataFrame(stops_trip_df, geometry=gpd.points_from_xy(stops_trip_df.stop_lon, stops_trip_df.stop_lat)).drop(columns=['stop_lat','stop_lon'])
stops_trip_gdf.head(10)

,trip_id,route_id,service_id,direction_id,stop_id,stop_sequence,geometry
29193,1791020,1,3,0,1407,2,POINT (-79.99205 40.44402)
52726,1791020,1,3,0,1408,3,POINT (-79.99645 40.44342)
50606,1791020,1,3,0,180,4,POINT (-79.99977 40.44254)
64536,1791020,1,3,0,213,5,POINT (-80.00074 40.44445)
97218,1791020,1,3,0,214,6,POINT (-80.00196 40.44785)
111637,1791020,1,3,0,215,7,POINT (-80.00278 40.45032)
173973,1791020,1,3,0,216,8,POINT (-80.00289 40.45303)
211469,1791020,1,3,0,1796,9,POINT (-80.00192 40.45322)
243825,1791020,1,3,0,2088,10,POINT (-80.00064 40.45345)
266746,1791020,1,3,0,2089,11,POINT (-79.99929 40.45370)


In [53]:
# https://stackoverflow.com/questions/22219004/how-to-group-dataframe-rows-into-list-in-pandas-groupby
#route_points_df['stop_sequence'] = stops_trip_gdf.groupby(['route_id', 'direction_id'])['stop_id'].apply(list).reset_index()
#route_points_df['point_sequence'] = stops_trip_gdf.groupby(['route_id', 'direction_id'])['geometry'].apply(list).reset_index()
route_points_df = stops_trip_gdf.groupby(['route_id', 'direction_id']).agg({'stop_id': lambda x: list(x), 'geometry': lambda x: list(x)}).reset_index()
route_points_df.head()

,route_id,direction_id,stop_id,geometry
0,1,0,"[1407, 1408, 180, 213, 214, 215, 216, 1796, 20...","[POINT (-79.992052 40.444022), POINT (-79.9964..."
1,1,1,"[20817, 16595, 21344, 12830, 12634, 12635, 126...","[POINT (-79.75711200000001 40.600684), POINT (..."
2,11,0,"[1407, 1408, 180, 213, 214, 215, 944, 19329, 2...","[POINT (-79.992052 40.444022), POINT (-79.9964..."
3,12,0,"[179, 180, 213, 214, 215, 216, 2596, 217, 219,...","[POINT (-79.998481 40.443888), POINT (-79.9997..."
4,12,1,"[318, 22320, 15587, 322, 323, 325, 20728, 2072...","[POINT (-80.030216 40.586668), POINT (-80.0411..."


In [54]:
# save and export as df
route_points_df.to_pickle('PT_route_data.csv')

In [59]:
#temp = route_points_df.loc[(route_points_df.route_id == '1') & (route_points_df.direction_id == 1)]['point_sequence']
route_points_df.dtypes

route_id        object
direction_id     int64
stop_id         object
geometry        object
dtype: object

In [60]:
stops_trips_df.head()

,trip_id,route_id,service_id,direction_id,stop_id,stop_sequence,stop_lat,stop_lon,route_dir_id
0,10517020,61C,3,1,22902,1,40.352322,-79.860598,61C_1
1,1959020,61C,3,1,22902,1,40.352322,-79.860598,61C_1
4,11433020,P7,3,1,22902,1,40.352322,-79.860598,P7_1
8,4274020,61C,3,1,22902,1,40.352322,-79.860598,61C_1
11,11227020,61C,3,1,22902,1,40.352322,-79.860598,61C_1


In [61]:
stops_trips_df['route_dir_id'] = stops_trips_df['route_id'] + '_' + stops_trips_df['direction_id'].astype(str)
#stops_trips_df.head(50)
route_dir_stops_df = stops_trips_df[['stop_id','route_dir_id', 'stop_lat', 'stop_lon']].groupby(['stop_id']).agg({'stop_lat': 'first', 'stop_lon': 'first', 'route_dir_id': lambda x: list(set(x))}).reset_index()
route_dir_stops_df.shape
route_dir_stops_df.head(10)
route_dir_stops_df.loc[route_dir_stops_df['stop_id'] == '1189']  # quick check

(6735, 4)

,stop_id,stop_lat,stop_lon,route_dir_id
0,10005,40.340754,-79.981175,[Y47_1]
1,10016,40.403958,-79.990230,"[51_0, 51L_0]"
2,10018,40.401127,-79.989317,"[51_0, 51L_0]"
3,10019,40.399887,-79.989153,"[51_0, 51L_0]"
4,10022,40.396720,-79.987743,"[51_0, 51L_0]"
5,10023,40.395140,-79.986830,"[51_0, 51L_0]"
6,10025,40.393302,-79.986562,[51_0]
7,10026,40.391477,-79.986685,"[51_0, 51L_0]"
8,10027,40.389384,-79.985526,"[51_0, 51L_0]"
9,10028,40.388190,-79.984822,[51_0]


,stop_id,stop_lat,stop_lon,route_dir_id
530,1189,40.459412,-79.944722,"[86_1, 64_1]"


In [62]:
route_dir_stops_df.to_pickle('stops_routes_dir.csv')  # for each stop, provides a list of associated route-dir pairs